# Organize and Clean OpenStreetMap and NYC Film Permits Data

In [1]:
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', None)

## Loading Data
Data acquisition in `osm_data.ipynb` and `film_to_data.ipynb` notebooks.  OSM data includes street name, borough of street, and geometry.  Permit data includes street names (main street, 1st cross street, 2nd cross street), zipcodes of production, and main borough of production.

In [2]:
# Load OSM GeoJSON
nyc = gpd.read_file('../data/nyc_osm.geojson')

nyc.head(3)

,name,boro,geometry
0,100th avenue,queens,"MULTILINESTRING ((-73.74175 40.71573, -73.7413..."
1,100th drive,queens,"MULTILINESTRING ((-73.73203 40.71458, -73.7319..."
2,100th place,queens,"LINESTRING (-73.81879 40.60576, -73.81889 40.6..."


In [3]:
# Load NYC Film Permits JSON
permits = pd.read_json('../data/permits.json')

permits.head(3)

,eventid,startdate,enddate,entered,boro,category,subcategory,country,zipcode,parkingheld,m_street,c1_street,c2_street
0,623629,2022-02-02,2022-02-03,2022-01-28,manhattan,Television,Episodic series,United States of America,"[10010, 10011, 10012, 10013]",prince street between bowery and lafayette street,prince street,bowery,lafayette street
1,623629,2022-02-02,2022-02-03,2022-01-28,manhattan,Television,Episodic series,United States of America,"[10010, 10011, 10012, 10013]",mott street between east houston street and pr...,mott street,east houston street,prince street
2,623629,2022-02-02,2022-02-03,2022-01-28,manhattan,Television,Episodic series,United States of America,"[10010, 10011, 10012, 10013]",mulberry street between east houston street an...,mulberry street,east houston street,prince street


## Regex Street Name Cleaning and Standardization

In [4]:
from re_clean_functions import make_mod_name, standardize_cardinal, standardize_way_type, ord_to_num, clean_saint

In [5]:
# Strip ordinal signifiers from street names for OSM streets
nyc['osm_name'] = nyc['name']
nyc['name'] = nyc['name'].map(lambda x: make_mod_name(x))

nyc.head(1)

,name,boro,geometry,osm_name
0,100 avenue,queens,"MULTILINESTRING ((-73.74175 40.71573, -73.7413...",100th avenue


In [6]:
# Drop permits with null street name values
permits = permits.loc[permits['m_street'].notnull()]

In [7]:
# Example of permit with ordinal street name
sample = permits.loc[permits['m_street'].str.contains('100th')].head(1)
sample

,eventid,startdate,enddate,entered,boro,category,subcategory,country,zipcode,parkingheld,m_street,c1_street,c2_street
16788,585489,2021-06-18,2021-06-18,2021-06-14,queens,Television,Cable-episodic,United States of America,"[11691, 11693]",beach 100th st between rockaway beach boulevar...,beach 100th st,rockaway beach boulevard,shore front pkwy


In [8]:
# Strip ordinal signifiers from street names for film permit streets
permits['m_street'] = permits['m_street'].map(lambda x: make_mod_name(x))
permits['c1_street'] = permits['c1_street'].map(lambda x: make_mod_name(x))
permits['c2_street'] = permits['c2_street'].map(lambda x: make_mod_name(x))

In [9]:
permits.iloc[sample.index[0]].reset_index().set_index('index').T

index,eventid,startdate,enddate,entered,boro,category,subcategory,country,zipcode,parkingheld,m_street,c1_street,c2_street
16788,585489,2021-06-18,2021-06-18,2021-06-14,queens,Television,Cable-episodic,United States of America,"[11691, 11693]",beach 100th st between rockaway beach boulevar...,beach 100 st,rockaway beach boulevard,shore front pkwy


In [10]:
# Example of street name with cardinal value
sample = permits.loc[permits['m_street'].str.contains('^w ')].head(1)
sample

,eventid,startdate,enddate,entered,boro,category,subcategory,country,zipcode,parkingheld,m_street,c1_street,c2_street
384,623016,2022-01-28,2022-01-29,2022-01-25,manhattan,Television,Cable-episodic,United States of America,"[10001, 10011, 10014]",w 14th st between 10 avenue and 9th ave,w 14 st,10 avenue,9 ave


In [11]:
# Convert cardinal abbreviation to full cardinal name (ex. 'w' -> 'west')
nyc['name'] = nyc['name'].map(lambda x: standardize_cardinal(x))
permits['m_street'] = permits['m_street'].map(lambda x: standardize_cardinal(x))
permits['c1_street'] = permits['c1_street'].map(lambda x: standardize_cardinal(x))
permits['c2_street'] = permits['c2_street'].map(lambda x: standardize_cardinal(x))

In [12]:
permits.iloc[sample.index[0]].reset_index().set_index('index').T

index,eventid,startdate,enddate,entered,boro,category,subcategory,country,zipcode,parkingheld,m_street,c1_street,c2_street
384,623016,2022-01-28,2022-01-29,2022-01-25,manhattan,Television,Cable-episodic,United States of America,"[10001, 10011, 10014]",w 14th st between 10 avenue and 9th ave,west 14 st,10 avenue,9 ave


In [13]:
# Convert abbreviation of street type to full name (ex. 'ave' -> 'avenue')
nyc['name'] = nyc['name'].map(lambda x: standardize_way_type(x))
permits['m_street'] = permits['m_street'].map(lambda x: standardize_way_type(x))
permits['c1_street'] = permits['c1_street'].map(lambda x: standardize_way_type(x))
permits['c2_street'] = permits['c2_street'].map(lambda x: standardize_way_type(x))

In [14]:
permits.iloc[sample.index[0]].reset_index().set_index('index').T

index,eventid,startdate,enddate,entered,boro,category,subcategory,country,zipcode,parkingheld,m_street,c1_street,c2_street
384,623016,2022-01-28,2022-01-29,2022-01-25,manhattan,Television,Cable-episodic,United States of America,"[10001, 10011, 10014]",w 14th st between 10 avenue and 9th ave,west 14 street,10 avenue,9 avenue


In [15]:
# Example of street with ordinal name
sample = permits.loc[permits['m_street'].str.contains('sixth')].head(1)
sample

,eventid,startdate,enddate,entered,boro,category,subcategory,country,zipcode,parkingheld,m_street,c1_street,c2_street
2133,616183,2021-12-16,2021-12-16,2021-12-13,manhattan,Commercial,Commercial,United States of America,"[10001, 10013]",sixth avenue between canal street and walker s...,sixth avenue,canal street,walker street


In [16]:
# Convert ordinal name to numerical value (ex. 'sixth' -> '6')
nyc['name'] = nyc['name'].map(lambda x: ord_to_num(x))
permits['m_street'] = permits['m_street'].map(lambda x: ord_to_num(x))
permits['c1_street'] = permits['c1_street'].map(lambda x: ord_to_num(x))
permits['c2_street'] = permits['c2_street'].map(lambda x: ord_to_num(x))

In [17]:
permits.iloc[sample.index[0]].reset_index().set_index('index').T

index,eventid,startdate,enddate,entered,boro,category,subcategory,country,zipcode,parkingheld,m_street,c1_street,c2_street
2133,616183,2021-12-16,2021-12-16,2021-12-13,manhattan,Commercial,Commercial,United States of America,"[10001, 10013]",sixth avenue between canal street and walker s...,6 avenue,canal street,walker street


In [18]:
# Convert 'st' or 'st.' to 'saint' (at beginning of name)
nyc['name'] = nyc['name'].map(lambda x: clean_saint(x))
permits['m_street'] = permits['m_street'].map(lambda x: clean_saint(x))
permits['c1_street'] = permits['c1_street'].map(lambda x: clean_saint(x))
permits['c2_street'] = permits['c2_street'].map(lambda x: clean_saint(x))

## Feature Extraction
In this section location data will be extracted from each row of OSM street data and each row of film permit data.

In [19]:
from re_find_functions import has_num, has_way_type, has_cardinal, get_way_name, is_film_boro

### Cardinal Values
Columns are created for cardinal values from the front, middle, and end of street names.

In [20]:
nyc['temp'] = nyc['name'].map(lambda x: has_cardinal(x))
nyc['front_card'] = nyc['temp'].map(lambda x: x[0])
nyc['middle_card'] = nyc['temp'].map(lambda x: x[1])
nyc['back_card'] = nyc['temp'].map(lambda x: x[2])

permits['temp'] = permits['m_street'].map(lambda x: has_cardinal(x))
permits['m_front_card'] = permits['temp'].map(lambda x: x[0])
permits['m_middle_card'] = permits['temp'].map(lambda x: x[1])
permits['m_back_card'] = permits['temp'].map(lambda x: x[2])

permits['temp'] = permits['c1_street'].map(lambda x: has_cardinal(x))
permits['c1_front_card'] = permits['temp'].map(lambda x: x[0])
permits['c1_middle_card'] = permits['temp'].map(lambda x: x[1])
permits['c1_back_card'] = permits['temp'].map(lambda x: x[2])

permits['temp'] = permits['c2_street'].map(lambda x: has_cardinal(x))
permits['c2_front_card'] = permits['temp'].map(lambda x: x[0])
permits['c2_middle_card'] = permits['temp'].map(lambda x: x[1])
permits['c2_back_card'] = permits['temp'].map(lambda x: x[2])

In [21]:
# Drop temp column
nyc.drop(columns='temp', inplace=True)
permits.drop(columns='temp', inplace=True)

### Number, Type, Name
- Columns are created for street numbers (ex. 'west 4 street' -> '4')
- Columns are created for 'way' type (ex. 'west 4 street' -> 'street')
- Columns are created for 'way' name (ex. 'prince street' -> 'prince')

In [22]:
nyc['st_num'] = nyc['name'].map(lambda x: has_num(x))
nyc['way_type'] = nyc['name'].map(lambda x: has_way_type(x))
nyc['way_name'] = nyc['name'].map(lambda x: get_way_name(x))

permits['m_st_num'] = permits['m_street'].map(lambda x: has_num(x))
permits['m_way_type'] = permits['m_street'].map(lambda x: has_way_type(x))
permits['m_way_name'] = permits['m_street'].map(lambda x: get_way_name(x))

permits['c1_st_num'] = permits['c1_street'].map(lambda x: has_num(x))
permits['c1_way_type'] = permits['c1_street'].map(lambda x: has_way_type(x))
permits['c1_way_name'] = permits['c1_street'].map(lambda x: get_way_name(x))

permits['c2_st_num'] = permits['c2_street'].map(lambda x: has_num(x))
permits['c2_way_type'] = permits['c2_street'].map(lambda x: has_way_type(x))
permits['c2_way_name'] = permits['c2_street'].map(lambda x: get_way_name(x))

### Possible Boroughs
In the case where a street cannot be found by matching the name and the listed main production borough, all zipcodes of the production will be checked and the street name will be checked against the those boroughs found (ex. main production borough is Manhattan but street name is 'flatbush avenue').

In [23]:
permits['bronx'] = permits['zipcode'].map(lambda x: is_film_boro(x, 'bronx'))
permits['brooklyn'] = permits['zipcode'].map(lambda x: is_film_boro(x, 'brooklyn'))
permits['manhattan'] = permits['zipcode'].map(lambda x: is_film_boro(x, 'manhattan'))
permits['queens'] = permits['zipcode'].map(lambda x: is_film_boro(x, 'queens'))
permits['staten_island'] = permits['zipcode'].map(lambda x: is_film_boro(x, 'staten island'))

In [24]:
permits.head(3)

,eventid,startdate,enddate,entered,boro,category,subcategory,country,zipcode,parkingheld,m_street,c1_street,c2_street,m_front_card,m_middle_card,m_back_card,c1_front_card,c1_middle_card,c1_back_card,c2_front_card,c2_middle_card,c2_back_card,m_st_num,m_way_type,m_way_name,c1_st_num,c1_way_type,c1_way_name,c2_st_num,c2_way_type,c2_way_name,bronx,brooklyn,manhattan,queens,staten_island
0,623629,2022-02-02,2022-02-03,2022-01-28,manhattan,Television,Episodic series,United States of America,"[10010, 10011, 10012, 10013]",prince street between bowery and lafayette street,prince street,bowery,lafayette street,None,None,None,None,None,None,None,None,None,None,street,prince,None,None,bowery,None,street,lafayette,False,False,True,False,False
1,623629,2022-02-02,2022-02-03,2022-01-28,manhattan,Television,Episodic series,United States of America,"[10010, 10011, 10012, 10013]",mott street between east houston street and pr...,mott street,east houston street,prince street,None,None,None,east,None,None,None,None,None,None,street,mott,None,street,houston,None,street,prince,False,False,True,False,False
2,623629,2022-02-02,2022-02-03,2022-01-28,manhattan,Television,Episodic series,United States of America,"[10010, 10011, 10012, 10013]",mulberry street between east houston street an...,mulberry street,east houston street,prince street,None,None,None,east,None,None,None,None,None,None,street,mulberry,None,street,houston,None,street,prince,False,False,True,False,False


In [25]:
nyc.head(3)

,name,boro,geometry,osm_name,front_card,middle_card,back_card,st_num,way_type,way_name
0,100 avenue,queens,"MULTILINESTRING ((-73.74175 40.71573, -73.7413...",100th avenue,None,None,None,100,avenue,None
1,100 drive,queens,"MULTILINESTRING ((-73.73203 40.71458, -73.7319...",100th drive,None,None,None,100,drive,None
2,100 place,queens,"LINESTRING (-73.81879 40.60576, -73.81889 40.6...",100th place,None,None,None,100,place,None


## Write GeoJSON and JSON

In [26]:
nyc.to_file('../data/nyc_clean.geojson', driver='GeoJSON')
permits.to_json('../data/permits_clean.json')